
# Advent of Code 2015

### In the style of Peter Norvig

Advent of code is programming contest, where you score points by being in the quickest 100 people to return a correct answer.  The programming language is your choice.  This is why I have kept time records in many of the puzzles.  To see how I compare against the winners, you can look at the daily Leaderboards, for eg, <a href=https://adventofcode.com/2015/leaderboard/day/12> day 12 </a>.<p>  
I selected 2015, because Peter Norvig, whose writes pretty code, has already published for 2016-2019 and 2020 isn't available yet.


First some imports and utility Functions.
Most of Peter's utility functions have moved into <a href=file:///Volumes/generic/apuri/g/misc/python/2015-advent-of-code/util.py> util.py </a>
### author: avp

## Preliminaries - all the days will need this to have executed first


In [2]:
import time
import itertools as it
import math
import collections
import re
import operator as op
import numpy
import sys
import string
import re
import functools as ft
import util as u
import more_itertools as mi
#from fastcore.test import test_eq  # fastcore is library I want to learn

def test_eq(actual,expected):  # glorified assert, but when it fails you get some useful info
    if actual != expected:
        print('actual=',actual)
        print('expected=', expected)
        assert False

def Input(n, dir = 'data'):
        return open(f'{dir}/{n:02d}.txt').read()

def remove_all(elt, xs):
        return [x for x in xs if x != elt]



## <a href="https://adventofcode.com/2015/day/1">Day 1 </a> :  Not quite Lisp 
<a href=https://adventofcode.com/2015/leaderboard/day/1> leaders </a>

In [ ]:
def bracketmap(bracketstr):
        return [1 if c == '(' else -1 for c in bracketstr ]

# tests
test_eq(list(map(u.compose(sum, bracketmap), '(())  ()()  (((  (()(()(  ))(((((  ())  ))(  )))  )())())'.split())),
       [0, 0, 3, 3, 3, -1, -1, -3, -3])

# puzzle
test_eq(sum(bracketmap(mi.first(u.Input(1)))), 138)

In [ ]:
# Part 2

def steps_to_basement(bracketstr):
        "2020.09.16 1905-2005 - mostly to learn doctest"
        return 2+mi.last(enumerate(it.takewhile(lambda floor: floor >= 0, 
                                                       it.accumulate(bracketmap(bracketstr)))),
                         (-1,0))[0] # the [0] picks out the enumeration number 

# tests
test_eq([steps_to_basement(s) for s in '()())  )'.split()], [5, 1])

# puzzle
test_eq(steps_to_basement(Input(1, 'data').strip()), 1771)


## <a href="https://adventofcode.com/2015/day/2"> Day 2 </a>: I Was Told There Would Be No Math
<a href=https://adventofcode.com/2015/leaderboard/day/2> leaders </a>

In [ ]:
def box_area_to_wrap(dims):
        """
        dims is a tuple of box dimensions (length l, width w, and height h)
        to wrap we need the surface area + slack = area of smallest side
        """
        sides = [x*y for x, y in it.combinations(dims, 2)]
        return sum(sides) * 2 + min(sides)

# tests
test_eq([box_area_to_wrap(dims) for dims in [(2,3,4), (1,1,10)]], 
        [58, 43])
 
# puzzle
test_eq(sum(map(box_area_to_wrap, u.Input(2, u.integers))), 1586300)


In [ ]:
# Part 2
def ribbon(dims):
        """
        2020.09.16 2005-2045

        A present with dimensions 2x3x4 needs 
        2+2+3+3 = 10 feet of ribbon to wrap the present plus 
        2*3*4 = 24 feet of ribbon for the bow, for a total of 34 feet.
        
        A present with dimensions 1x1x10 needs 
        1+1+1+1 = 4 feet of ribbon to wrap the present plus 
        1*1*10 = 10 feet of ribbon for the bow, for a total of 14 feet.
       """
        dims = sorted(dims)
        return sum(dims[:2]) * 2 + math.prod(dims)
    
# tests
test_eq(list(map(ribbon, [(2,3,4),(1,1,10)])),  [34, 14])

# puzzle
test_eq(sum(ribbon(dims) for dims in u.Input(2, u.integers)), 3737498)


   

## <a href="https://adventofcode.com/2015/day/3"> Day 3 </a>: Perfectly Spherical Houses in a Vacuum
<a href=https://adventofcode.com/2015/leaderboard/day/3> leaders </a>

In [ ]:
P = complex

def pathremap(path):
        "convert the arrows into 'steps' in the complex plane"
        d = {   '^' : P(0,1), '>' : P(1),
                'v' : P(0,-1), '<': P(-1) }
        return map(d.get, path)

def distinct_points(path):
        "2020.09.16 2045-2105 = 20m"
        return len(set(it.accumulate(it.chain([P(0)], pathremap(path)))))


test_eq([distinct_points(s) for s in ['>', '^>v<', '^v^v^v^v^v']],
        [2, 4, 2])
test_eq(distinct_points(mi.first(u.Input(3))), 2565)


In [ ]:
def two_santas(path):
        "2020.09.15 2105-2135 = 30m"
        paths = mi.distribute(2, pathremap(path))  # one path for each santa
        paths = [it.accumulate(it.chain([P(0)],path)) for path in paths]
        return len(set(it.chain(*paths)))

test_eq([two_santas(s) for s in ['^v', '^>v<', '^v^v^v^v^v']],
    [3, 3, 11])
test_eq(two_santas(mi.first(u.Input(3))), 2639)


## <a href="https://adventofcode.com/2015/day/4"> Day 4 </a>: The Ideal Stocking Stuffer
<a href=https://adventofcode.com/2015/leaderboard/day/4> leaders </a>

These can take a /while/ to evaluate 

In [ ]:
import hashlib

def functional_find_num(key, nzeros=5):
        # NOT USED : just here to show that sometimes fp seems less clear
        return mi.first(i
            for i, m in enumerate(map(u.compose(hashlib.md5,
                                            str.encode, 
                                            ft.partial(op.add, key), 
                                                    # prepend key
                                            str),   # convert count to a str
                                  it.count()))
                if m.hexdigest()[:nzeros] == '0'*nzeros)

                
def find_num(key, nzeros=5):
        "2020.09.16 2135-2200 = 25m"
        for i in it.count():
                s = key+str(i)
                m = hashlib.md5(s.encode())
                if m.hexdigest()[:nzeros] == '0'*nzeros: return i

       
# tests
test_eq(u.mapt(find_num, ['abcdef', 'pqrstuv']),
        (609043, 1048970))

# puzzle
%time test_eq(find_num('iwrupvqb'), 346386)

In [ ]:
# takes about 20s on my machine
%time test_eq(find_num('iwrupvqb', 6), 9958218)

## <a href="https://adventofcode.com/2015/day/5"> Day 5 </a>: Doesn't He Have Intern-Elves For This?
<a href=https://adventofcode.com/2015/leaderboard/day/5> leaders </a>


In [ ]:
def is_nice(s):
        "2020.09.16 2200-2230 = 30m"
        def n_vowels(s):
                return len([c for c in s if c in 'aeiou'])
        def has_double_letter(s):
                return next((x for x, y in mi.pairwise(s) if x == y), None) != None
        def has_forbiddens(s):
                forbiddens = 'ab cd pq xy'.split()
                return any(bad in s for bad in forbiddens)
        return n_vowels(s) >= 3 and has_double_letter(s) and not has_forbiddens(s)

# tests
test_eq(list(map(is_nice, 'ugknbfddgicrmopn aaa jchzalrnumimnmhp haegwjzuvuyypxyu dvszwmarrgswjxmb'.split())),
        [True, True, False, False, False])

# puzzle
test_eq(mi.quantify(map(is_nice,u.Input(5))), 258)

In [ ]:
def is_new_nice(s):
        "2020.09.16 2230-2245    15m"
        def has_double_letter_with_gap(s):
                for i in range(len(s)-2):
                        if s[i] == s[i+2]:
                                return True
                return False
        def has_duplicate_nonoverlapping_pair(s):
                for i in range(len(s)-3):
                        if s[i:i+2] in s[i+2:]:
                                return True
                return False
        return has_duplicate_nonoverlapping_pair(s) and has_double_letter_with_gap(s)

# tests
test_eq(list(map(is_new_nice, 'xyxy qjhvhtzxzqqjkmpb xxyxx uurcxstgmygtbstg ieodomkazucvgmuy'.split())),
        [True, True, True, False, False])
# puzzle
test_eq(mi.quantify(map(is_new_nice,u.Input(5))), 53)

## <a href="https://adventofcode.com/2015/day/6"> Day 6 </a>: Probably a Fire Hazard
<a href=https://adventofcode.com/2015/leaderboard/day/6> leaders </a>


In [ ]:
def get_cmd(words):
        if words[0] == 'toggle':
                words = ['.'] + list(words)
        _, cmd, tlx, tly, _, brx, bry = words
        tlx, tly, brx, bry = map(int,(tlx, tly, brx, bry))
        return cmd, tlx, tly, brx+1, bry+1

def nlights(instructions):
        # 2020.09.16 2300-2330    30m
        sz = 1000
        L = [False]*sz*sz

        for words in instructions:
                cmd, tlx, tly, brx, bry = get_cmd(words)
                for x in range(tlx, brx):
                        for y in range(tly, bry):
                                pt = x + y*sz
                                if cmd == 'toggle':
                                        L[pt] = not L[pt]
                                else:
                                        L[pt] = cmd == 'on'
        return mi.quantify(L)

#assert brightness(instructions) == 14110788
# puzzle
%time test_eq(nlights(u.Input(6, u.words)), 377891)


In [ ]:
# this needs get_cmd from the cell above
def brightness(instructions):
        # 2020.09.16 2330-2350  20m - accidentally dropped +P(1,1)
        sz = 1000
        L = [0]*sz*sz

        for words in instructions:
                cmd, tlx, tly, brx, bry = get_cmd(words)
                for x in range(tlx, brx):
                        for y in range(tly, bry):
                                pt = x + y*sz
                                if cmd == 'on':
                                        L[pt] += 1
                                elif cmd == 'off':
                                        L[pt] = max(0, L[pt] - 1)
                                else:
                                        L[pt] += 2
        return sum(L)


# puzzle
%time test_eq(brightness(u.Input(6, u.words)), 14110788)


## <a href="https://adventofcode.com/2015/day/7"> Day 7 </a>: Some Assembly Required
<a href=https://adventofcode.com/2015/leaderboard/day/7> leaders </a>

In [ ]:
def get_emulated(instructions, initial_registers={}):
        # 2020.09.17    0815-1030
        #               stuck on numpy.uint16, then introducing "pending" dic

        skipB           = initial_registers != {}
        integer_type    = numpy.uint16
        registers       = initial_registers
        pending         = collections.defaultdict(list)
        letters         = re.compile('^[a-z]+$')
        digits          = re.compile('^[0-9]+$')
        ops             = {     'AND'   : op.and_,
                                'OR'    : op.or_,
                                'LSHIFT': op.lshift,
                                'RSHIFT': op.rshift,
                                'NOT'   : op.invert,      }

        def eval(x):
                return registers[x] if letters.match(x) else integer_type(x)

        def is_ready(x):
                return  (letters.match(x) == None or x in registers)

        def process(instruction, skipB):
                words = instruction.split()
                if words[1] == '->':
                        # assignment
                        a, _, tgt = words
                        if not is_ready(a):
                                pending[a].append(instruction)
                                return

                        if not (skipB and tgt == 'b'):
                                registers[tgt] = eval(a)
                elif words[2] == '->':
                        # unary op
                        op, a, _, tgt = words
                        if not is_ready(a):
                                pending[a].append(instruction)
                                return
                        if not (skipB and tgt == 'b'):
                                registers[tgt] = ops[op](eval(a))
                elif words[3] == '->':
                        # binary op
                        a, op, b, _, tgt = words
                        if not is_ready(a):
                                pending[a].append(instruction)
                                return
                        if not is_ready(b):
                                pending[b].append(instruction)
                                return
                        if not (skipB and tgt == 'b'):
                                registers[tgt] = ops[op] (eval(a), eval(b))
                else:
                        print(words)
                        assert False

                # process any instructions pending on tgt being ready
                instructions, pending[tgt] = pending[tgt], []
                for instruction in instructions:
                        process(instruction, skipB)


        for instruction in instructions:
                process(instruction, skipB)

        return registers['a']  # return the value of register 'a'

#instructions = Input(7, 'data').strip().split('\n')
instructions = u.Input(7)

# puzzle - part 1
test_eq(get_emulated(instructions), 3176) 

# puzzle - part 2
test_eq(get_emulated(instructions, { 'b': 3176 }), 14710)


## <a href="https://adventofcode.com/2015/day/8"> Day 8 </a>: Matchsticks
<a href=https://adventofcode.com/2015/leaderboard/day/8> quickest </a>

In [ ]:
def spacecalc(xs):
        # 2020.09.17    2100-2110       10m
        return sum(len(x) - len(eval(x)) for x in xs)

test_eq(spacecalc(u.Input(8)), 1350)



In [ ]:
def space2calc(xs):
        #       looked for built in escape for ages before realizing it is trivial
        def escape(str):
                cs = ['"']
                for c in str:
                        if c in '"\\':
                                cs.append('\\')
                        cs.append(c)
                cs.append('"')
                return ''.join(cs)

        return sum(len(escape(x)) - len(x) for x in xs)

test_eq(space2calc(u.Input(8)), 2085)
 


## <a href="https://adventofcode.com/2015/day/9"> Day 9 </a>: All in a Single Night
<a href=https://adventofcode.com/2015/leaderboard/day/9> leaders </a>


In [ ]:
def shortest(distances, func = min):
        # 2020.09.17    2135-2215       40m
        # 2020.09.17    2215-2217        2m
        cities = set(it.chain(*[[x[0],x[2]] for x in distances]))  # x[0] is from city, x[2] is to city
        distances = dict((tuple(sorted((x[0], x[2]))), int(x[3])) for x in distances)
        routes = list(it.permutations(cities))
        return func(sum (map(u.compose(distances.get,   # get distance of ordered pair of cities
                                       tuple,           # distances dict key needs a tuple, not a list
                                       sorted),         # order city pairs, because distances dict is 
                             mi.pairwise(route)         # take each city pair along the route
                        ))                              # total route distance
                    for route in routes
        )


distances = u.Input(9, u.words)

# puzzles
test_eq(shortest(distances), 251)
test_eq(shortest(distances, max), 898)



## <a href="https://adventofcode.com/2015/day/10"> Day 10 </a>: Elves Look, Elves Say
<a href=https://adventofcode.com/2015/leaderboard/day/10> leaders </a>


In [ ]:
def lookandsay(digits, ntimes):
        # 2020.09.17    2220-2240       20m
        for i in range(ntimes):
                digits = u.cat(str(len(tuple(g))) + k 
                               for k, g in it.groupby(digits))
        return digits

# tests
test_eq(lookandsay('1',1), '11')
test_eq(lookandsay('1',2), '21')
test_eq(lookandsay('1',3), '1211')
test_eq(lookandsay('1',4), '111221')
test_eq(lookandsay('1',5), '312211')

%time test_eq(len(lookandsay('1113122113', 40)), 360154)


In [ ]:
# puzzle part 2 - separated because almost 8s elapse to complete
%time test_eq(len(lookandsay('1113122113', 50)), 5103798)


## <a href="https://adventofcode.com/2015/day/11"> Day 11 </a>: Corporate Policy
<a href=https://adventofcode.com/2015/leaderboard/day/11> leaders </a>


In [ ]:
def NOTUSED_has_pairs(xs):
        "interpret 'different' to mean the pairs can't be for the same letter"
        pairs = [x if x == y else False for x, y in zip(xs, xs[1:])]
        return len(set(x for x in pairs if x)) > 1

def has_pairs(xs):
        # the clunky return is to support skip_non_pairs
        pair_locs = [i for i, (x, y) in enumerate(mi.pairwise(xs)) if x == y]
        if len(pair_locs) > 2: return True, pair_locs   # if 3 or more pairs, 2 must be nonoverlapping
        if len(pair_locs) < 2: return False, pair_locs  # don't have two pairs
        x, y = pair_locs  # have exactly two pairs
        return y - x > 1, pair_locs  # that do not overlap

test_eq(has_pairs(''), (False, []))
test_eq(list(map(has_pairs, 'aabb aaaa aabaa aabcc aaa aa aabc abc'.split())),
        [(True, [0, 2]), (True, [0, 1, 2]), (True, [0, 3]), (True, [0, 3]), (False, [0, 1]), (False, [0]), (False, [0]), (False, [])])

def has_sequence(letters):
        nums = [ord(c) for c in letters]
        nums = [nums[i+1] - nums[i] for i in range(len(nums)-1)]
        for i in range(len(nums)-1):
                if nums[i] == 1 and nums[i+1] == 1:
                        return True
        return False

test_eq(list(map(has_sequence, 'ab abc abcd abd dab dabc'.split() )), 
        [False, True, True, False, False, True])
test_eq(has_sequence(''), False)


def gen_password(password):
        """
        2020.09.18                      2h10
                        0810-0835         25m   finding baseconvert and getting password incrementing working
                        0835-0900         25m   make baseconvert use tuples rather than base26 strings
                        0900-0920         20m   implemented exclude iol rule efficiently
                        0920-0945         25m   run_of_3 letters in sequence check
                                                implemented exclude iol rule efficiently
                                                by eliminating from allowable letters and using base 23
                        0955-1030         35m   wrote 2 pair checkers for each meaning of 'different'
                        1030-1100         30m   initialize to handle any excluded password chars

        baseconvert uses
        0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
        """

        from baseconvert import base

        def mkmaps(excluded):
                pairs = list(enumerate(c for c in string.ascii_lowercase if not c in excluded))
                return dict(pairs), dict((y, x) for x, y in pairs)

        def L_to_num(letters, effective_base):
                return int(base(tuple(map(L_to_n.get, letters)), 
                                effective_base, 10, string=True))

        def num_to_L(num, effective_base):
                return tuple(map(n_to_L.get, base(num, 10, effective_base)))
            
        def lpad(field, width, letter='a'):
            return [letter]*(width-len(field)) + list(field)

        def next_without_excluded(password, exclude, effective_base):
                "ensure password has no excluded letters"
                def next(letter):
                        return chr(ord(letter)+1)
                # find the earliest excluded letter
                i = mi.first((i for i, c in enumerate(password) if c in exclude), None)
                if i == None:
                        return password

                # the code below assumes that excluded letters are not adjacent and don't include z
                password = password[:i] + next(password[i]) + 'a' * (len(password) - i - 1)
                assert excluded not in password
                n = L_to_num(password, effective_base)
                return num_to_L(n-1, effective_base)
        
        def skip_non_pairs(letters, effective_base=23):
                # an optimisation function which reduced puzzle 2 time from 20s to 5s 
                penult, ult = tuple(map(L_to_n.get, letters[-2:]))
                skip = penult - ult if ult < penult else effective_base - ult
                return skip
    
        excluded = 'ilo'  # see note in initialize
        effective_base = 26 - len(excluded)
        n_to_L, L_to_n = mkmaps(excluded)

        password = next_without_excluded(password, excluded, effective_base)
        n = 1 +L_to_num(password, effective_base)
        for i in it.count():   # just so we have an iteration count
                letters = lpad(num_to_L(n, effective_base), len(password))
                has_p, pair_locs = has_pairs(letters)
                if has_sequence(letters) and has_p:
                        return u.cat(letters)
                # skip to next, or if we are sure there are no pairs, skip ahead
                n += 1 if len(pair_locs) != 0 else skip_non_pairs(letters, effective_base)
        assert False

test_eq(gen_password('abcdefgh'), 'abcdffaa')
test_eq(gen_password('ghijklmn'), 'ghjaabcc')


%time test_eq(gen_password('hxbxwxba'), 'hxbxxyzz')

In [ ]:
# part 2 - puzzle - separated because it took 20s on my machine before optimization
%time test_eq(gen_password('hxbxxyzz'), 'hxcaabcc')


## <a href="https://adventofcode.com/2015/day/11"> Day 11 </a>: Corporate Policy : Try 2 

<a href=https://adventofcode.com/2015/leaderboard/day/11> leaders </a>

The base version is long and fiddly.  Let's try again without baseconvert.<p>
That was satisfying:  less code, 2x speed


In [ ]:
def threeples(xs):
    return zip(xs, xs[1:], xs[2:])

def gen_password(pwd):
        def nextv(s):        
                exclusions = list(s.index(x) for x in excluded if x in s)
                if len(exclusions) == 0:
                        return s
                pos = min(exclusions)
                ## XXX assumes exclusions are not contiguous
                return s[:pos+1] + u.cat(['z']*(len(s)-pos-1))

        def increv(s):
                "add one to the number represented backwards"
                return [chr(ord(s[0])+1)] + s[1:] if s[0] != 'z' else ['a'] + increv(s[1:])

        def n_pairs(s):
                "return the number of pairs - but don't bother to look for more than 2"
                return len(tuple(mi.take(2, (1 for c in valid if c+c in s))))

        def has_seq(s):
                x=string.ascii_lowercase
                has_seq.seq3 = getattr(has_seq, 'seq3', 
                                       set(z for z in threeples(x)
                                           if not any(excl in z for excl in excluded)))
                return any(x in has_seq.seq3 for x in threeples(s))

        excluded = 'ilo'
        valid = str(sorted(set(string.ascii_lowercase) - set(excluded)))

        pwd = nextv(pwd)
        for i in it.count():
                pwd = u.cat(list(reversed(increv(list(reversed(pwd))))))
                n = n_pairs(pwd)
                if n >= 2 and has_seq(pwd):
                        return u.cat(pwd)
                if n==0: # optimisation to skip things that couldn't be valid
                        penult, ult = pwd[-2:]
                        c = 'z' if penult < ult else pwd[-2]
                        pwd = pwd[:-1] + c


test_eq(list(map(has_seq, 'ab abc abcd abd dab dabc abcdffaa'.split() )), 
        [False, True, True, False, False, True, True])
test_eq(has_seq(''), False)

test_eq(has_two_pairs([]), (False))
test_eq(list(map(has_two_pairs, 'aabb aaaa aabaa aabcc aaa aa aabc abc abcdffaa'.split())),
        [True, False, False, True, False, False, False, False, True])

    
test_eq(gen_password('abcdefgh'), 'abcdffaa')
test_eq(gen_password('ghijklmn'), 'ghjaabcc')
%time test_eq(gen_password('hxbxwxba'), 'hxbxxyzz')
%time test_eq(gen_password('hxbxxyzz'), 'hxcaabcc')


## <a href="https://adventofcode.com/2015/day/12"> Day 12 </a>: JSAbacusFramework.io
<a href=https://adventofcode.com/2015/leaderboard/day/12> leaders </a>


In [ ]:
jsons = mi.first(u.Input(12))
test_eq(sum(u.integers(jsons)), 119433)

In [ ]:
import json
def subsummer(jo, skip):
        "return a json object with any dicts containing <skip> omitted"
        if isinstance(jo, int):
                return jo
        if isinstance(jo, str):  
                # this isn't needed in practice, but seem to be needed by problem statement
                return sum(u.integers(jo))
        
        if isinstance(jo, dict) and skip not in jo.values():
                xs = jo.values()
        elif isinstance(jo, list):
                xs = jo
        else:
                return 0

        return sum(subsummer(o, skip) for o in xs)


test_eq(subsummer(json.loads(jsons), 'red'), 68466)

## <a href="https://adventofcode.com/2015/day/13"> Day 13 </a>: Knights of the Dinner Table
<a href=https://adventofcode.com/2015/leaderboard/day/13> leaders </a>


In [ ]:
def table_seating(happiness_table, add_self = False):
        """
        2020.09.19      0115-0200          45m  - cyclic permutations
                        0200-0215          15m  - add self
        """
        def cyclicpermutations(people):
                "XXX - we also don't care whether we cycle cw or anti-cw but both appear here"
                return ([people[0]] + list(permutation) 
                        for permutation in it.permutations(people[1:]))

        def score(cycle):
                return sum(scores[(x,y)] + scores[(y,x)] 
                           for x, y in mi.pairwise(cycle + [cycle[0]]))

        def get_scores(happiness_table):
                return dict(((row[0], row[-1]), 
                             int(row[3]) * (1 if row[2] == 'gain' else -1 ))
                            for row in happiness_table)

        scores = get_scores(happiness_table)
        people = set(it.chain(*scores.keys()))
        if add_self:
                for p in people:
                        scores[('self', p)] = scores[(p, 'self')] = 0
                people.add('self')
        return max(map(score, cyclicpermutations(list(people))))

# tests
happiness = u.tInput(13, u.words)
test_eq(table_seating(happiness, add_self=False), 330)

# puzzle part 1
happiness = u.Input(13, u.words)
test_eq(table_seating(happiness, add_self=True), 725)

#puzzle part 2
test_eq(table_seating(happiness, add_self=False), 733)



## <a href="https://adventofcode.com/2015/day/14"> Day 14 </a>: Reindeer Olympics 
<a href=https://adventofcode.com/2015/leaderboard/day/14> leaders </a>


In [84]:
def get_speed_rest(speed_rest_table):
        return [(row[0], *u.integers(u.cat(row[1:])))
                for row in speed_rest_table]
                        
def distance(seconds, speed, flies, rests):
                chunks, seconds = divmod(seconds, flies + rests)
                seconds = min(seconds, flies)
                return chunks * speed * flies + speed * seconds

def furthest(speed_rest_table, seconds):
        "2020.09.19      0220-0245          25m  - whaaa - felt like i was much quicker"
        return max(distance(seconds, *x[1:]) for x in get_speed_rest(speed_rest_table))

def distancepoints(speed_rest_table, seconds):
        "2020.09.19      0245-0310          25m  - whaaa - felt like i was much quicker"
        speed_rest_table = get_speed_rest(speed_rest_table)
        scores = collections.defaultdict(int)
        for secs in range(1, seconds+1):
                winning_distance = max(distance(secs, *x[1:]) for x in speed_rest_table)
                winners = ( x[0] for x in speed_rest_table if winning_distance == distance(secs, *x[1:]) )
                for winner in winners:
                        scores[winner] += 1
        return max(scores.values())

# puzzle 1
test_eq(furthest(u.Input(14, u.words), 2503), 2660)

# tests
test_eq([furthest(u.tInput(14, u.words), seconds) 
         for seconds in [1, 10, 11, 1000]],
        [16,160,176,1120])

# puzzle 2
test_eq(distancepoints(u.Input(14, u.words), 2503), 1256)

## <a href="https://adventofcode.com/2015/day/15"> Day 15 </a>: Science for Hungry People 
<a href=https://adventofcode.com/2015/leaderboard/day/15> leaders </a>


In [ ]:
def cookie(ntsps, ingredients, desiredcals=None):
        """
        2020.09.19      0325-0340       15m     - just to get the input!!
                        0340-0400       20m     - got score working
        """
        def score(proportions):
                "a feature is a capacity, durability, flavor or texture"
                feature_sums = [sum(ingredients[ingr][feature] * ntsps
                                        for ingr, ntsps in enumerate(proportions))
                                for feature in range(1, 5)]
                return 0 if any(x <= 0 for x in feature_sums) else math.prod(feature_sums)

        def calories(proportions):
                cals = 5        # calories is the 5th element of an ingredient
                return sum(ingredients[ingr][cals] * ntsps
                           for ingr, ntsps in enumerate(proportions))

        def get_ingredients(ingredients):
                "return [ingredient, capacity, durability, flavor, texture, calories]"
                return [(row.split(':')[0], *u.integers(u.cat(row)))
                        for row in ingredients]

        def combos(tot, n):
                "return all combinations of n numbers that sum to tot"
                if n == 1: yield [tot]
                else:      
                        for i in range(tot//n+1):
                                yield from ([i] + c for c in combos(tot - i, n-1))

        def all_proportions(ntsps, ningredients):
                for combo in combos(ntsps, ningredients):
                        yield from set(it.permutations(combo))

        ingredients = get_ingredients(ingredients)
        return max(map(score, all_proportions(ntsps, len(ingredients))))    if desiredcals == None else \
               max(map(score, [a for a in all_proportions(ntsps, len(ingredients)) 
                               if calories(a) == desiredcals]))

ingredients = u.Input(15)
%time test_eq(cookie(100, ingredients, 500), 11171160)
%time test_eq(cookie(100, ingredients), 13882464)

## <a href="https://adventofcode.com/2015/day/16"> Day 16 </a>: Aunt Sue
<a href=https://adventofcode.com/2015/leaderboard/day/16> leaders </a>

In [ ]:
def whichsue(suefacts, want, checks):
        """
        2020.09.19      1050-1105       15m     - read q, get input, setup
                        1105-1130       25m     - implemented filters
                        1130-1135        5m     - implemented altered filters (puzzle 2)
        """
        def get_facts(suefacts):
                def words_which_are_not_numbers(words):
                    return [w for w in words if len(u.integers(w)) == 0]
                
                return [dict(zip(words_which_are_not_numbers(u.words(facts))[1:],
                                 u.integers(facts)[1:]))
                        for facts in suefacts]

        def is_sue_possible(sue):
                return all(checks.get(fact_name, op.eq) (suefacts[sue][fact_name], fact_value)
                           for fact_name, fact_value in want.items()
                           if fact_name in suefacts[sue])

        suefacts = get_facts(suefacts)
        return 1 + mi.first(filter(is_sue_possible, range(len(suefacts))))

suefacts = u.Input(16)
want = {'children': 3,
        'cats': 7,
        'samoyeds': 2,
        'pomeranians': 3,
        'akitas': 0,
        'vizslas': 0,
        'goldfish': 5,
        'trees': 3,
        'cars': 2,
        'perfumes': 1,
}

# puzzle 1
test_eq(whichsue(suefacts, want, {}), 213)

# puzzle 2
inequality_checks = {
        'cats'        : op.gt,
        'trees'       : op.gt,
        'pomeranians' : op.lt,
        'goldfish'    : op.lt,
}
test_eq(whichsue(suefacts, want, inequality_checks), 323)

## <a href="https://adventofcode.com/2015/day/17"> Day 17 </a>: No Such Thing as Too Much
<a href=https://adventofcode.com/2015/leaderboard/day/17> leaders </a>


In [ ]:
def eggnog(nliters, container_sizes, fewest_containers = False):
        """
        2020.09.19      1035-1140        5m     - read problem, got input
                        1140-1210       30m     - working on recursion ... tired
                        1435-1515       40m     - got recursion working
                        1515-1545       30m     - got fewest_containers bit working,
                                                  tidied up into assertions for posterity
        """

        def allot(nliters, containers, prefix = []):
                "returns a list of allotments"
                if nliters == 0:
                        return [prefix]
                if  nliters < 0:
                        return None     # impossible
                allotments = []
                for i in range(len(containers)):
                        skip = containers[i]
                        x = allot(nliters - skip, containers[i+1:], prefix + [skip])
                        if x != None:
                                allotments.extend(x)
                return allotments

        container_sizes = tuple(sorted(container_sizes, reverse=True))
        allotments = allot(nliters, container_sizes)
        if fewest_containers:
                n = min(map(len, allotments))
                allotments = [a for a in allotments if len(a) == n]
        return len(allotments)


# tests 1
container_sizes = u.integers(Input(17,'test'))
assert 4 == eggnog(25, container_sizes)

# puzzle 1
container_sizes = u.integers(Input(17,'data'))
assert 1638 == eggnog(150, container_sizes)

# tests 2
container_sizes = u.integers(Input(17,'test'))
assert eggnog(25, container_sizes, fewest_containers = True) == 3

# puzzle 2
container_sizes = u.integers(Input(17,'data'))
assert eggnog(150, container_sizes, fewest_containers = True) == 17

## <a href="https://adventofcode.com/2015/day/18"> Day 18 </a>: Like a GIF For Your Yard
<a href=https://adventofcode.com/2015/leaderboard/day/18> leaders </a>

In [62]:
def animate_lights(nsteps, grid_rows, force_corners=False):
        """
        2020.09.20      2335-2400       25m     - read problem, got input
        2020.09.21      0550-0600       10m     - p2
        """
        def load(grid_rows):
                grid = collections.defaultdict(u.Point)
                for y, row in enumerate(grid_rows):
                        for x in range(len(row.strip())):
                                p = u.Point(x,y)
                                grid[p] = row[x] == '#'
                return len(row), len(grid_rows), grid

        def reprgrid(w, h, grid):
                lines = []
                for y in range(h):
                        lines.append(u.cat('#' if grid[u.Point(x,y)] else '.' for x in range(w)))
                return u.cat(lines)

        def step(w, h, grid):
                new_grid = collections.defaultdict(u.Point)
                for y in range(h):
                        for x in range(w):
                                pt = u.Point(x,y)
                                n = sum(grid[p] for p in u.neighbors8(pt))
                                new_grid[pt] = n == 2 or n == 3 if grid[pt] else  n == 3
                return new_grid

        def force(w, h, grid, force_corners):
                if force_corners:
                        for x, y in ( (0,0), (0,h-1), (w-1,h-1), (w-1,0) ):
                                grid[u.Point(x, y)] = True
                return grid


        w, h, grid = load(grid_rows)
        grid = force(w, h, grid, force_corners)
        for i in range(nsteps):
#               print(reprgrid(w,h,grid))
                grid = step(w, h, grid)
                grid = force(w, h, grid, force_corners)
#               print()
        return sum(grid.values())


# puzzle 1
%time test_eq(animate_lights(100, grid_rows), 821)

# puzzle 2
grid_rows = u.Input(18, file_template='test/{:02d}a.txt')
test_eq(animate_lights(5, grid_rows, force_corners=True), 17)
grid_rows = u.Input(18)
%time test_eq(animate_lights(100, grid_rows, force_corners=True), 886)

CPU times: user 5.95 s, sys: 99.1 ms, total: 6.05 s
Wall time: 6.27 s
CPU times: user 5.69 s, sys: 80.7 ms, total: 5.77 s
Wall time: 5.92 s


### Speed up?
Can animate_lights go faster if we represent grid as a list instead of a defaultdict?
Now we need to explicitly ensure there is enough space for neighbors

In [83]:
def animate_lights(nsteps, grid_rows, force_corners=False):
        def load(grid_rows):
                grid, ymax = [], len(grid_rows)
                for y, row in enumerate(grid_rows):
                        xs, xmax = [], len(row.strip())
                        for x in range(xmax):
                                xs.append(row[x] == '#')
                        grid.append([False] + xs + [False]) # has borders
                grid = [[False]*(2+xmax)] + grid + [[False]*(2+xmax)]   # has borders
                return xmax, ymax, list(mi.flatten(grid))  

        def P(x,y,w):  return (y+1)*(w+2)+x+1  # w+2, y+1 allow for borders

        def reprgrid(w, h, grid):
                lines = []
                for y in range(h):
                        p = P(0,y,w)
                        lines.append(u.cat('#' if grid[p+x] else '.' for x in range(w)))
                return u.cat(lines)

        def nbor8_offsets(w):
                return  (-w-3, -w-2, -w-1,
                         -1,         +1,
                          w+1,  w+2,  w+3)

        def step(w, h, grid):
                new_grid = [False]*len(grid)
                for y in range(h):
                        p0 = P(0,y,w)
                        for x in range(w):
                                p = p0 + x
                                n = sum(grid[p+d] for d in nbor8)
                                new_grid[p] = n == 2 or n == 3 if grid[p] else  n == 3
                return new_grid

        def force(grid, gridcorners):
                if gridcorners:
                        for p in gridcorners:
                                grid[p] = True
                return grid
        
        w, h, grid = load(grid_rows)

        # Precompute offsets of eight neighboring squares.
        nbor8 = nbor8_offsets(w)
        gridcorners = []
        if force_corners:
            gridcorners = [P(x,y,w) for x, y in ( (0,0), (0,h-1), (w-1,h-1), (w-1,0) )]

        grid = force(grid, gridcorners)

        for i in range(nsteps):
#               print(reprgrid(w,h,grid))
                grid = step(w, h, grid)
                grid = force(grid, gridcorners)
        return sum(grid)


# puzzle 1
%time test_eq(animate_lights(100, grid_rows), 821)

# puzzle 2
grid_rows = u.Input(18, file_template='test/{:02d}a.txt')
test_eq(animate_lights(5, grid_rows, force_corners=True), 17)
grid_rows = u.Input(18)
%time test_eq(animate_lights(100, grid_rows, force_corners=True), 886)

CPU times: user 2.08 s, sys: 37.3 ms, total: 2.12 s
Wall time: 2.2 s
CPU times: user 2.05 s, sys: 32.4 ms, total: 2.08 s
Wall time: 2.14 s


In [69]:
#!/usr/bin/env python3

import sys

def on(board, i, j):
    if i < 0 or i >= 100 or j < 0 or j >= 100: return False
    return board[i][j] == "#"

def evolve(board):
    newboard = [["."]*100 for i in range(100)]

    for i in range(100):
        for j in range(100):
            count = 0
            for a in range(-1,2):
                for b in range(-1,2):
                    if not (a == 0 and b == 0):
                        if on(board, i+a, j+b):
                            count += 1
            if on(board, i, j):
                if count == 2 or count == 3:
                    newboard[i][j] = '#'
                else:
                    newboard[i][j] = '.'
            else:
                if count == 3:
                    newboard[i][j] = '#'
                else:
                    newboard[i][j] = '.'

    return newboard


def main(fname):
    board = [[c for c in s.strip()] for s in open(fname)]

    for i in range(100):
        board = evolve(board)

    count = 0
    for a in board:
        for b in a:
            if b == '#':
                count += 1
    print(count)

%time main('data/18.txt')

821
CPU times: user 5.79 s, sys: 84.2 ms, total: 5.87 s
Wall time: 6.02 s


## <a href="https://adventofcode.com/2015/day/19"> Day 19 </a>: Medicine for Rudolph
<a href=https://adventofcode.com/2015/leaderboard/day/19> leaders </a>